# Collecting Data From NBA API

In [73]:
# Import Libraries
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.static import players
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [82]:
# Get Player IDs of all Players

all_players = players.get_active_players()

print(len(all_players))

571


In [ ]:
# Get the career avg data of all players

df = pd.DataFrame()
iteration = 1

for player in all_players:
    try:
        print(f'Iteration: {iteration}')
        player_id = player.get('id', None)

        time.sleep(2)
    
        if player_id is not None:
            player_object = playercareerstats.PlayerCareerStats(per_mode36= 'PerGame', player_id= str(player_id))
            df_player = player_object.get_data_frames()[0]
            df = pd.concat([df, df_player])
    
    except KeyError as e: print(f'Skipping Player ID: {player_id}')
    except Exception as e: print(f'Trouble getting player {player_id}')

    iteration += 1
 
df
df.to_csv('../Datasets/nba.csv')

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15


/var/folders/kp/0yh04nk14gq26qb__8kstckh0000gn/T/ipykernel_19190/1814564361.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_player])


Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65
Iteration: 66
Iteration: 67
Iteration: 68
Iteration: 69
Iteration: 70
Iteration: 71
Iteration: 72
Iteration: 73
Iteration: 74
Iteration: 75
Iteration: 76
Iteration: 77
Iteration: 78
Iteration: 79
Iteration: 80
Iteration: 81
Iteration: 82
Iteration: 83
Iteration: 84
Iteration: 85
Iteration: 86
Iterat

In [ ]:
# Tried to retrieve the failed data but the dataframes wer empty

failed_df = pd.DataFrame()
failed_ids = ["1642385", "1641752", "1642261", "1628995", "1642278", "1631132", "1629723", "1628436", "1628398", "1631222"]
double_failed_ids = []

for id in failed_ids:
    print(f'Trying {id}')
    try:
        player_object = playercareerstats.PlayerCareerStats(per_mode36= 'PerGame', player_id= id)
        player_df = player_object.get_data_frames()[0]
        pd.concat([failed_df, player_df])

    except KeyError as e: print(f'Skipping Player ID: {player_id}')
    except Exception as e: print(f'Trouble getting player {player_id}')

    time.sleep(2)

failed_df

Trying 1642385
Skipping Player ID: 203967
Trying 1641752
Trying 1642261
Trying 1628995
Trying 1642278
Trying 1631132
Trying 1629723
Trying 1628436
Trying 1628398
Trying 1631222


""


In [88]:
df = pd.read_csv('../Datasets/nba.csv')

df

,Unnamed: 0,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,0,1630173,2020-21,0,1610612748,MIA,21.0,61,4,12.1,...,0.509,1.2,2.2,3.4,0.5,0.3,0.5,0.7,1.5,5.0
1,1,1630173,2021-22,0,1610612761,TOR,22.0,73,28,23.6,...,0.595,2.0,4.5,6.5,1.1,0.5,0.6,1.2,2.1,9.1
2,2,1630173,2022-23,0,1610612761,TOR,23.0,55,12,20.7,...,0.702,1.8,4.1,6.0,0.9,0.6,0.5,1.1,1.9,9.2
3,3,1630173,2023-24,0,1610612761,TOR,24.0,25,0,17.5,...,0.571,2.0,3.4,5.4,1.8,0.6,0.5,1.2,1.6,7.7
4,4,1630173,2023-24,0,1610612752,NYK,24.0,49,18,24.2,...,0.643,2.9,4.3,7.2,1.1,0.6,1.1,1.1,2.1,7.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3945,8,203967,2022-23,0,1610612760,OKC,29.0,20,0,13.7,...,0.844,0.7,2.6,3.3,0.9,0.4,0.1,1.0,1.5,7.4
3946,9,203967,2022-23,0,0,TOT,29.0,57,12,14.1,...,0.829,0.9,2.7,3.6,1.3,0.4,0.1,1.0,1.7,6.4
3947,10,203967,2023-24,0,1610612744,GSW,30.0,64,9,17.2,...,0.849,1.1,3.3,4.4,2.3,0.5,0.2,1.2,1.8,8.0
3948,11,203967,2024-25,0,1610612743,DEN,31.0,16,4,13.1,...,0.700,0.9,2.3,3.1,1.4,0.4,0.1,0.9,1.2,3.5
